In [1]:
execfile('/home/chris/forums_scraper/classes/scraper.py')

In [2]:
fs = ForumsScraper()

In [3]:
fs.get_top150()

In [4]:
fs.profile_list

[('https://community.tableau.com/s/profile/0054T000001NuFj', 'Others'),
 ('https://community.tableau.com/s/profile/0054T000001Namx', 'Others'),
 ('https://community.tableau.com/s/profile/0054T000001NiRN', 'Others')]

In [5]:
profiles = []
print("Retrieving all selected profiles")
for profile_url, profile_type in fs.profile_list[0:5]:
    prof_dict = fs.get_profile(profile_url, profile_type)
    profiles.append(prof_dict)

Retrieving all selected profiles


In [6]:
import datetime

print("Converting data to DataFrame")
df = pd.DataFrame(profiles)
# Replace Johan DeGroots ID
df["id"] = df["id"].replace("0054T000001NkSe","XX54T000001NkSe")
df["snapshot"] = datetime.datetime.now().date()
df["snapshot"] = pd.to_datetime(df["snapshot"]).date()

Converting data to DataFrame


AttributeError: 'Series' object has no attribute 'date'

In [9]:
datetime.datetime.now().date()

datetime.date(2025, 1, 19)

In [10]:
print("Converting data to DataFrame")
df = pd.DataFrame(profiles)
# Replace Johan DeGroots ID
df["id"] = df["id"].replace("0054T000001NkSe","XX54T000001NkSe")
df["snapshot"] = datetime.datetime.now().date()

Converting data to DataFrame


In [11]:
df.head()

,id,points,Following,Followers,Posts,Comments,Likes Received,snapshot
0,0054T000001NuFj,5658,0,5,15,321,51,2025-01-19
1,0054T000001Namx,12977,0,1,6,1542,9,2025-01-19
2,0054T000001NiRN,80948,8,45,30,4977,1137,2025-01-19


In [12]:
print("Getting existing snapshots from Google Sheets")
snapshot_sheet = fs.get_gsheet('https://docs.google.com/spreadsheets/d/1H86_fj8z8yBNKOZOIBGrxc8-hTc03XZxNxYBzTneo7w/edit',
            0)
snapshot_data = pd.DataFrame(snapshot_sheet.get_all_records())
# Ensure the `snapshot` column in the Google Sheets data is also in `date` format
snapshot_data["snapshot"] = pd.to_datetime(snapshot_data["snapshot"]).dt.date

Getting existing snapshots from Google Sheets


In [13]:
snapshot_data.head()

,id,snapshot,points,Following,Followers,Posts,Comments,Likes Received
0,0054T000001NiRN,2023-04-21,11142,2,9,21,622,111
1,0054T000001NiRN,2023-04-22,11162,2,9,21,624,112
2,0054T000001NiRN,2023-04-23,11234,2,9,21,631,113
3,0054T000001NiRN,2023-04-24,11279,2,9,21,634,116
4,0054T000001NiRN,2023-04-25,11315,2,9,21,635,119


In [14]:
import pandas as pd
print("Concatenating new snapshot with existing data")
df_new = pd.concat([df,snapshot_data])
df_new = df_new.sort_values(by=["id","snapshot","points"]).drop_duplicates(subset=["id","snapshot"],keep="last")
# print(df_new)
for col in ["Following","Followers","Posts","Comments","Likes Received"]:
    df_new[col] = pd.to_numeric(df_new[col], errors='coerce').fillna(0).astype(int)
df_new = df_new.sort_values(by=["id","snapshot"]).set_index(["id","snapshot"])

Concatenating new snapshot with existing data


In [15]:
print("Concatenating new snapshot with existing data")
df_new = pd.concat([df,snapshot_data])
df_new["snapshot"] = pd.to_datetime(df_new["snapshot"]).dt.date
df_new = df_new.sort_values(by=["id","snapshot","points"]).drop_duplicates(subset=["id","snapshot"],keep="last")
# print(df_new)
for col in ["Following","Followers","Posts","Comments","Likes Received"]:
    df_new[col] = pd.to_numeric(df_new[col], errors='coerce').fillna(0).astype(int)
df_new = df_new.sort_values(by=["id","snapshot"]).set_index(["id","snapshot"])

Concatenating new snapshot with existing data


In [16]:
df_new.head()

points  Following  Followers  Posts  Comments  \
id              snapshot                                                   
0054T000001Namx 2023-04-21  12976          0          1      6      1542   
                2023-04-22  12976          0          1      6      1542   
                2023-04-23  12976          0          1      6      1542   
                2023-04-24  12976          0          1      6      1542   
                2023-04-25  12976          0          1      6      1542   

                            Likes Received  
id              snapshot                    
0054T000001Namx 2023-04-21               6  
                2023-04-22               6  
                2023-04-23               6  
                2023-04-24               6  
                2023-04-25               6

In [17]:
print("Creating multi-index in new data")
start_date = datetime.datetime(year=2023, month=4, day=21)
all_dates = pd.date_range(start_date, datetime.datetime.now().date(), freq='D')
new_index = pd.MultiIndex.from_product([df_new.index.levels[0],all_dates],names=["id","snapshot"])
new_df = pd.DataFrame(index=new_index)

df_new = new_df.merge(df_new,left_index=True,right_index=True,how='left').reset_index()

Creating multi-index in new data


In [18]:
df_new.head()

,id,snapshot,points,Following,Followers,Posts,Comments,Likes Received
0,0054T000001Namx,2023-04-21,12976,0,1,6,1542,6
1,0054T000001Namx,2023-04-22,12976,0,1,6,1542,6
2,0054T000001Namx,2023-04-23,12976,0,1,6,1542,6
3,0054T000001Namx,2023-04-24,12976,0,1,6,1542,6
4,0054T000001Namx,2023-04-25,12976,0,1,6,1542,6


In [19]:
print("Interpolating values for any missing dates")

df_list = []

for idx in df_new.id.unique():
    dft = df_new[df_new["id"] == idx].copy()  # Create a copy to avoid SettingWithCopyWarning
    for col in dft.columns:
        if col in ["points", "Following", "Followers", "Comments", "Likes Received", "Posts"]:
            dft.loc[:, col] = dft[col].interpolate(method="linear", limit_area="inside")
        else:
            dft.loc[:, col] = dft[col].bfill()  # Use bfill directly instead of fillna
    df_list.append(dft)

df_new = pd.concat(df_list)

Interpolating values for any missing dates


/tmp/ipykernel_117143/4061733004.py:9: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  dft.loc[:, col] = dft[col].interpolate(method="linear", limit_area="inside")


In [20]:
        print("Final tidying up")
        # Alter Johan De Groot's ID
        df_new["id"] = df_new["id"].replace("0054T000001NkSe","XX54T000001NkSe")
        df_new = df_new[[x for x in df_new.columns if "Unnamed" not in x]]

        print("Writing new DataFrame back to Google Sheets")
        snapshot_sheet.clear()
        set_with_dataframe(snapshot_sheet,df_new)

        fs.chrome_driver.close()

Final tidying up
Writing new DataFrame back to Google Sheets
